# Assignment 8: Data Aggregation and Group Operations

## Overview
This assignment covers data aggregation and group operations using healthcare/insurance EHR data. You'll analyze patient encounters (insurance claims), healthcare providers, and medical facilities across different regions. The data includes provider statistics, facility information, and encounter charges with insurance coverage details.
id
## Setup

In [16]:
# TODO: Load the three CSV files into dataframes
# provider_df = pd.read_csv(...)
# facility_df = pd.read_csv(...)
# encounter_df = pd.read_csv(...)
provider_df = pd.read_csv('data/provider_data.csv')
facility_df = pd.read_csv('data/facility_data.csv')
encounter_df = pd.read_csv('data/encounter_data.csv')

print(provider_df.columns)
print(facility_df.columns)
print(encounter_df.columns)

# TODO: Print the shapes of each dataframe
# print("Provider data shape:", ...)
print("Provider data shape:", provider_df.shape)
# print("Facility data shape:", ...)
print("Facility data shape:", facility_df.shape)
# print("Encounter data shape:", ...)
print("Encounter data shape:", encounter_df.shape)

# TODO: Merge the dataframes for analysis
# Hint: Start by merging encounter_df with provider_df on 'provider_id'
#       Then merge the result with facility_df on 'facility_id'
# Note: Both encounter_df and provider_df have 'facility_id', and both encounter_df and facility_df have 'region'
#       After merging, you may need to handle duplicate column names (pandas will add suffixes like _x and _y)
#       You can drop one of the duplicate columns and rename if needed
merged_df = pd.merge(
    encounter_df,
    provider_df.drop(columns=['facility_id']),  # drop to avoid conflict
    on='provider_id')

merged_df = pd.merge(
    merged_df,
    facility_df,
    on='facility_id')



# TODO: Print the merged dataframe shape, columns, and first few rows
# print("\nMerged data shape:", ...)
# print("\nColumns:", ...)
# print("\nFirst few rows:")
# print(...)
print("\nMerged data shape:", merged_df.shape)
print("\nColumns:", merged_df.columns.tolist())
print("\nFirst few rows:")
print(merged_df.head())


Index(['provider_id', 'provider_name', 'provider_type', 'facility_id',
       'specialty', 'years_experience', 'license_number'],
      dtype='object')
Index(['facility_id', 'facility_name', 'facility_type', 'region', 'beds',
       'established_date'],
      dtype='object')
Index(['encounter_id', 'patient_id', 'provider_id', 'facility_id',
       'encounter_date', 'encounter_type', 'diagnosis_code', 'procedure_code',
       'service_charge', 'insurance_paid', 'patient_paid', 'region'],
      dtype='object')
Provider data shape: (500, 7)
Facility data shape: (10, 6)
Encounter data shape: (5000, 12)

Merged data shape: (5000, 22)

Columns: ['encounter_id', 'patient_id', 'provider_id', 'facility_id', 'encounter_date', 'encounter_type', 'diagnosis_code', 'procedure_code', 'service_charge', 'insurance_paid', 'patient_paid', 'region_x', 'provider_name', 'provider_type', 'specialty', 'years_experience', 'license_number', 'facility_name', 'facility_type', 'region_y', 'beds', 'established_date

### Part 1.2: Basic Aggregation

**TODO: Perform basic groupby operations**

In [17]:
# TODO: Group by facility (healthcare facility) and calculate basic stats
grouped = merged_df.groupby('facility_name')
# TODO: Calculate mean, sum, count for years_experience by facility
grouped_experience = grouped['years_experience'].agg(['mean', 'sum', 'count']).rename(columns={
    'mean': 'avg_years_experience',
    'sum': 'total_years_experience',
    'count': 'total_providers'
})

# TODO: Calculate total encounter charges by facility
total_charges = grouped['service_charge'].sum().rename('total_service_charge')
# TODO: Calculate total insurance payments by facility
total_insurance_paid = grouped['insurance_paid'].sum().rename('total_insurance_paid')
# TODO: Find the facility with highest total encounter charges
highest_charge_facility = total_charges.idxmax()
highest_charge_amount = total_charges.max()
print(f"\nFacility with highest total encounter charges: {highest_charge_facility} (${highest_charge_amount:.2f})")

# TODO: Create a summary dataframe with one row per facility showing:
#       facility_name, total_providers, avg_years_experience, total_service_charge, total_insurance_paid
summary_df = pd.DataFrame({
    'total_providers': grouped_experience['total_providers'],
    'avg_years_experience': grouped_experience['avg_years_experience'],
    'total_service_charge': total_charges,
    'total_insurance_paid': total_insurance_paid
}).reset_index()

# TODO: Save the summary dataframe as 'output/q1_groupby_analysis.csv'
summary_df.to_csv('output/q1_groupby_analysis.csv', index=False)


Facility with highest total encounter charges: Pediatric Care Center ($361811.04)


### Part 1.3: Transform Operations

**TODO: Use transform operations to add group statistics**

In [18]:
# TODO: Add facility mean years_experience as new column
facility_mean_experience = merged_df.groupby('facility_name')['years_experience'].transform('mean')
merged_df['facility_mean_years_experience'] = facility_mean_experience

# TODO: Add facility standard deviation of years_experience
facility_std_experience = merged_df.groupby('facility_name')['years_experience'].transform('std')
merged_df['facility_std_years_experience'] = facility_std_experience

# TODO: Create normalized years_experience (z-score within facility)
merged_df['normalized_years_experience'] = (
    (merged_df['years_experience'] - merged_df['facility_mean_years_experience']) /
    merged_df['facility_std_years_experience']
)

# TODO: Add facility total encounter charges as new column
facility_total_charges = merged_df.groupby('facility_name')['service_charge'].transform('sum')
merged_df['facility_total_encounter_charges'] = facility_total_charges

# TODO: Display the enhanced dataframe
print("\nEnhanced Merged DataFrame with Aggregations:")
print(merged_df.head())
# TODO: Save results as 'output/q1_aggregation_report.txt'
with open('output/q1_aggregation_report.txt', 'w') as f:
    f.write(merged_df.to_string())#!/usr/bin/env python3


Enhanced Merged DataFrame with Aggregations:
  encounter_id patient_id provider_id facility_id encounter_date  \
0     ENC00001   PAT01564      PR0259      FAC004     2023-08-21   
1     ENC00002   PAT00296      PR0258      FAC001     2023-09-11   
2     ENC00003   PAT01381      PR0213      FAC001     2023-03-20   
3     ENC00004   PAT01131      PR0146      FAC004     2023-09-03   
4     ENC00005   PAT01388      PR0354      FAC003     2023-06-14   

         encounter_type diagnosis_code  procedure_code  service_charge  \
0          Office Visit          M79.3           99213          345.03   
1             Inpatient          N18.9           85025          550.48   
2                   Lab          K21.9           80053          149.84   
3  Outpatient Procedure          H52.9           70450         1468.48   
4             Emergency          G43.9           99214          672.23   

   insurance_paid  ...  license_number            facility_name facility_type  \
0          290.33  

## Question 2: Advanced GroupBy Operations

### Part 2.1: Filter Operations

**TODO: Use filter operations to remove groups**

In [ ]:
# TODO: Filter facilities with more than 20 providers (sufficient staffing)
# TODO: Filter facilities with average years_experience > 10 (experienced staff)
# TODO: Filter facilities with total encounter charges > 250000 (high volume)

# TODO: Create a summary of filtered results
# TODO: Save results as 'output/q2_filter_analysis.csv'

### Part 2.2: Apply Operations

**TODO: Use apply operations with custom functions**

In [ ]:
# TODO: Create custom function to calculate provider statistics
def provider_stats(group):
    # TODO: Return mean, std, min, max, range for years_experience
    pass

# TODO: Apply custom function to each facility
# Note: Add include_groups=False to avoid pandas FutureWarning about groupby columns
facility_provider_stats = merged_df.groupby('facility_name').apply(provider_stats, include_groups=False)

# TODO: Create function to find top providers in each facility
def top_providers(group, n=2):
    """Return top n providers in a group by encounter volume"""
    # TODO: Return top n rows by encounter count, selecting only provider_id, provider_name, 
    #       years_experience, specialty
    pass

# TODO: Apply function to get top 2 providers per facility
# IMPORTANT: When using groupby().apply(), the result has a MultiIndex where:
# - Level 0: The grouping column (facility_name)
# - Level 1: The original row index from each group
# Use reset_index(level=0) to move only the grouping column back to a regular column
# DO NOT use reset_index(drop=True) - that would drop ALL index levels including facility_name!
top_providers_by_facility = merged_df.groupby('facility_name').apply(
    lambda x: top_providers(x, n=2)
).reset_index(level=0)  # Moves facility_name from index to column

# TODO: Save results as 'output/q2_performance_report.txt'

### Part 2.3: Hierarchical Grouping

**TODO: Perform multi-level grouping**

In [ ]:
# TODO: Group by facility type and region
# TODO: Calculate statistics for each facility_type-region combination
# TODO: Use unstack to convert to wide format
# TODO: Use stack to convert back to long format

# TODO: Analyze the hierarchical structure
# TODO: Save results as 'output/q2_hierarchical_analysis.csv'

## Question 3: Pivot Tables and Cross-Tabulations

### Part 3.1: Basic Pivot Tables

**TODO: Create pivot tables for multi-dimensional analysis**

In [ ]:
# TODO: Create pivot table: encounter charges by procedure code and region
# TODO: Create pivot table with multiple aggregations (sum, mean, count)
# TODO: Add totals (margins) to pivot table
# TODO: Handle missing values with fill_value

# Tip: When saving a pivot table to CSV, the index (row labels) is preserved by default.
#      This is usually desired - when reading back, use index_col=0 to restore the index.
# TODO: Save results as 'output/q3_pivot_analysis.csv'

### Part 3.2: Cross-Tabulations

**TODO: Create cross-tabulations for categorical analysis**

In [ ]:
# TODO: Create crosstab of facility type vs region with counts
# TODO: Create crosstab with margins (row and column totals)
# TODO: Create multi-dimensional crosstab showing facility type × encounter type × region combinations
#       (Hint: use pd.crosstab with multiple index/column parameters, or create manually from the data)

# TODO: Analyze the cross-tabulation results
# TODO: Save results as 'output/q3_crosstab_analysis.csv'

### Part 3.3: Pivot Table Visualization

**TODO: Create visualizations from pivot tables**

In [ ]:
# TODO: Create heatmap from pivot table
# TODO: Create bar chart from pivot table
# TODO: Customize colors and styling
# TODO: Add appropriate titles and labels

# TODO: Save the plot as 'output/q3_pivot_visualization.png'

## Submission Checklist

Before submitting, verify you've created:

- [ ] `output/q1_groupby_analysis.csv` - Basic groupby analysis
- [ ] `output/q1_aggregation_report.txt` - Aggregation report
- [ ] `output/q2_filter_analysis.csv` - Filter operations analysis
- [ ] `output/q2_hierarchical_analysis.csv` - Hierarchical analysis
- [ ] `output/q2_performance_report.txt` - Performance report
- [ ] `output/q3_pivot_analysis.csv` - Pivot table analysis
- [ ] `output/q3_crosstab_analysis.csv` - Cross-tabulation analysis
- [ ] `output/q3_pivot_visualization.png` - Pivot visualization

## Key Learning Objectives

- Master the split-apply-combine paradigm
- Apply aggregation functions and transformations
- Create pivot tables for multi-dimensional analysis
- Apply advanced groupby techniques